In [390]:
import pandas as pd
import numpy as np

In [391]:
# webscrapping to get data related to movies in 2018

In [392]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2018"

In [393]:
df1 = pd.read_html(link, header=0)[2]
df2 = pd.read_html(link, header=0)[3]
df3 = pd.read_html(link, header=0)[4]
df4 = pd.read_html(link, header=0)[5]

In [394]:
df1

,Opening,Opening.1,Title,Production company,Cast and crew,.mw-parser-output .tooltip-dotted{border-bottom:1px dotted;cursor:help}Ref.
0,JANUARY,5,Insidious: The Last Key,Universal Pictures / Blumhouse Productions / S...,Adam Robitel (director); Leigh Whannell (scree...,[2]
1,JANUARY,5,The Strange Ones,Vertical Entertainment,Lauren Wolkstein (director); Christopher Radcl...,[3]
2,JANUARY,5,Stratton,Momentum Pictures,"Simon West (director); Duncan Falconer, Warren...",[4]
3,JANUARY,10,Sweet Country,Samuel Goldwyn Films,"Warwick Thornton (director); David Tranter, St...",[5]
4,JANUARY,12,The Commuter,Lionsgate / StudioCanal / The Picture Company,Jaume Collet-Serra (director); Byron Willinger...,[6]
...,...,...,...,...,...,...
69,MARCH,29,Ready Player One,Warner Bros. Pictures / Village Roadshow Pictu...,"Steven Spielberg (director); Ernest Cline, Zak...",[71]
70,MARCH,30,Tyler Perry's Acrimony,Lionsgate / Tyler Perry Films,Tyler Perry (director/screenplay); Taraji P. H...,[72]
71,MARCH,30,God's Not Dead: A Light in Darkness,Pure Flix,Michael Mason (director/screenplay); David A. ...,[73]
72,MARCH,30,Gemini,Neon,"Aaron Katz (director/screenplay); Lola Kirke, ...",[74]


In [395]:
data = df1.append(df2.append(df3.append(df4,ignore_index=True),ignore_index=True),ignore_index=True)

In [396]:
data

,Opening,Opening.1,Title,Production company,Cast and crew,.mw-parser-output .tooltip-dotted{border-bottom:1px dotted;cursor:help}Ref.,Ref.
0,JANUARY,5,Insidious: The Last Key,Universal Pictures / Blumhouse Productions / S...,Adam Robitel (director); Leigh Whannell (scree...,[2],NaN
1,JANUARY,5,The Strange Ones,Vertical Entertainment,Lauren Wolkstein (director); Christopher Radcl...,[3],NaN
2,JANUARY,5,Stratton,Momentum Pictures,"Simon West (director); Duncan Falconer, Warren...",[4],NaN
3,JANUARY,10,Sweet Country,Samuel Goldwyn Films,"Warwick Thornton (director); David Tranter, St...",[5],NaN
4,JANUARY,12,The Commuter,Lionsgate / StudioCanal / The Picture Company,Jaume Collet-Serra (director); Byron Willinger...,[6],NaN
...,...,...,...,...,...,...,...
265,DECEMBER,25,Holmes & Watson,Columbia Pictures / Gary Sanchez Productions,Etan Cohen (director/screenplay); Will Ferrell...,NaN,[163]
266,DECEMBER,25,Vice,Annapurna Pictures / Plan B Entertainment,Adam McKay (director/screenplay); Christian Ba...,NaN,[137]
267,DECEMBER,25,On the Basis of Sex,Focus Features,Mimi Leder (director); Daniel Stiepleman (scre...,NaN,[226]
268,DECEMBER,25,Destroyer,Annapurna Pictures,"Karyn Kusama (director); Phil Hay, Matt Manfre...",NaN,[258]


In [397]:
# now i will be using tmdb api to get details about the selected movie from the above list

In [398]:
!pip install tmdbv3api

In [399]:
from tmdbv3api import TMDb
import json
import requests

In [400]:
tmdb = TMDb()
tmdb.api_key = ''

In [401]:
from tmdbv3api import Movie
tmdb_movie = Movie()
def get_genre(x):
    genres = []
    result = tmdb_movie.search(x)
    if not result:
      return np.nan
    movie_id = result[0].id
    response = requests.get('https://api.themoviedb.org/3/movie/{}?api_key={}'.format(movie_id,tmdb.api_key))
    data_json = response.json()
    if data_json['genres']:
        genre_str = " " 
        for i in range(0,len(data_json['genres'])):
            genres.append(data_json['genres'][i]['name'])
        return genre_str.join(genres)
    else:
        np.NaN

In [402]:
data['genres'] = data['Title'].map(lambda x: get_genre(str(x)))

In [403]:
data.head()

,Opening,Opening.1,Title,Production company,Cast and crew,.mw-parser-output .tooltip-dotted{border-bottom:1px dotted;cursor:help}Ref.,Ref.,genres
0,JANUARY,5,Insidious: The Last Key,Universal Pictures / Blumhouse Productions / S...,Adam Robitel (director); Leigh Whannell (scree...,[2],NaN,Horror Mystery Thriller
1,JANUARY,5,The Strange Ones,Vertical Entertainment,Lauren Wolkstein (director); Christopher Radcl...,[3],NaN,Thriller Drama
2,JANUARY,5,Stratton,Momentum Pictures,"Simon West (director); Duncan Falconer, Warren...",[4],NaN,Action Thriller
3,JANUARY,10,Sweet Country,Samuel Goldwyn Films,"Warwick Thornton (director); David Tranter, St...",[5],NaN,Drama History Western
4,JANUARY,12,The Commuter,Lionsgate / StudioCanal / The Picture Company,Jaume Collet-Serra (director); Byron Willinger...,[6],NaN,Action Thriller


In [404]:
data.drop(columns='.mw-parser-output .tooltip-dotted{border-bottom:1px dotted;cursor:help}Ref.',inplace=True)

In [405]:
data.head()

,Opening,Opening.1,Title,Production company,Cast and crew,Ref.,genres
0,JANUARY,5,Insidious: The Last Key,Universal Pictures / Blumhouse Productions / S...,Adam Robitel (director); Leigh Whannell (scree...,NaN,Horror Mystery Thriller
1,JANUARY,5,The Strange Ones,Vertical Entertainment,Lauren Wolkstein (director); Christopher Radcl...,NaN,Thriller Drama
2,JANUARY,5,Stratton,Momentum Pictures,"Simon West (director); Duncan Falconer, Warren...",NaN,Action Thriller
3,JANUARY,10,Sweet Country,Samuel Goldwyn Films,"Warwick Thornton (director); David Tranter, St...",NaN,Drama History Western
4,JANUARY,12,The Commuter,Lionsgate / StudioCanal / The Picture Company,Jaume Collet-Serra (director); Byron Willinger...,NaN,Action Thriller


In [406]:
data_2018 = data[['Title','Cast and crew','genres']]
data_2018

,Title,Cast and crew,genres
0,Insidious: The Last Key,Adam Robitel (director); Leigh Whannell (scree...,Horror Mystery Thriller
1,The Strange Ones,Lauren Wolkstein (director); Christopher Radcl...,Thriller Drama
2,Stratton,"Simon West (director); Duncan Falconer, Warren...",Action Thriller
3,Sweet Country,"Warwick Thornton (director); David Tranter, St...",Drama History Western
4,The Commuter,Jaume Collet-Serra (director); Byron Willinger...,Action Thriller
...,...,...,...
265,Holmes & Watson,Etan Cohen (director/screenplay); Will Ferrell...,Mystery Adventure Comedy Crime
266,Vice,Adam McKay (director/screenplay); Christian Ba...,Thriller Science Fiction Action Adventure
267,On the Basis of Sex,Mimi Leder (director); Daniel Stiepleman (scre...,Drama History
268,Destroyer,"Karyn Kusama (director); Phil Hay, Matt Manfre...",Thriller Crime Drama Action


In [407]:
data_2018['Cast and crew'][1]

'Lauren Wolkstein (director); Christopher Radcliff (co-director/screenplay); Alex Pettyfer, James Freedson-Jackson, Emily Althaus, Gene Jones, Owen Campbell, Tobias Campbell'

In [408]:
def get_director(x):
    if " (director)" in x:
        return x.split(" (director)")[0]
    elif " (directors)" in x:
        return x.split(" (directors)")[0]
    else:
        return x.split(" (director/screenplay)")[0]

In [409]:
data_2018['director_name'] = data_2018['Cast and crew'].map(lambda x:get_director(x))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [410]:
data_2018

,Title,Cast and crew,genres,director_name
0,Insidious: The Last Key,Adam Robitel (director); Leigh Whannell (scree...,Horror Mystery Thriller,Adam Robitel
1,The Strange Ones,Lauren Wolkstein (director); Christopher Radcl...,Thriller Drama,Lauren Wolkstein
2,Stratton,"Simon West (director); Duncan Falconer, Warren...",Action Thriller,Simon West
3,Sweet Country,"Warwick Thornton (director); David Tranter, St...",Drama History Western,Warwick Thornton
4,The Commuter,Jaume Collet-Serra (director); Byron Willinger...,Action Thriller,Jaume Collet-Serra
...,...,...,...,...
265,Holmes & Watson,Etan Cohen (director/screenplay); Will Ferrell...,Mystery Adventure Comedy Crime,Etan Cohen
266,Vice,Adam McKay (director/screenplay); Christian Ba...,Thriller Science Fiction Action Adventure,Adam McKay
267,On the Basis of Sex,Mimi Leder (director); Daniel Stiepleman (scre...,Drama History,Mimi Leder
268,Destroyer,"Karyn Kusama (director); Phil Hay, Matt Manfre...",Thriller Crime Drama Action,Karyn Kusama


In [411]:
def get_actor1(x):
    return ((x.split("screenplay); ")[-1]).split(", ")[0])

In [412]:
data_2018['actor_1_name'] = data_2018['Cast and crew'].map(lambda x: get_actor1(x))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [413]:
def get_actor2(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 2:
        return 'unknown'
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[1])

In [414]:
def get_actor3(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 3:
        return 'unknown'
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[2])

In [415]:
data_2018['actor_2_name'] = data_2018['Cast and crew'].map(lambda x: get_actor2(x))
data_2018['actor_3_name'] = data_2018['Cast and crew'].map(lambda x: get_actor3(x))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [416]:
data_2018 = data_2018.rename(columns={'Title':'movie_title'})

In [417]:
final_data = data_2018[['director_name','actor_1_name','actor_2_name','actor_3_name','genres','movie_title']]

In [418]:
final_data.head()

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title
0,Adam Robitel,Lin Shaye,Angus Sampson,Leigh Whannell,Horror Mystery Thriller,Insidious: The Last Key
1,Lauren Wolkstein,Alex Pettyfer,James Freedson-Jackson,Emily Althaus,Thriller Drama,The Strange Ones
2,Simon West,Dominic Cooper,Austin Stowell,Gemma Chan,Action Thriller,Stratton
3,Warwick Thornton,Bryan Brown,Sam Neill,unknown,Drama History Western,Sweet Country
4,Jaume Collet-Serra,Liam Neeson,Vera Farmiga,Patrick Wilson,Action Thriller,The Commuter


In [419]:
# converting all str to lower
final_data['director_name'] = final_data['director_name'].str.lower()
final_data['actor_1_name'] = final_data['actor_1_name'].str.lower()
final_data['actor_2_name'] = final_data['actor_2_name'].str.lower()
final_data['actor_3_name'] = final_data['actor_3_name'].str.lower()
final_data['genres'] = final_data['genres'].str.lower()
final_data['movie_title'] = final_data['movie_title'].str.lower()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

In [420]:
final_data

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title
0,adam robitel,lin shaye,angus sampson,leigh whannell,horror mystery thriller,insidious: the last key
1,lauren wolkstein,alex pettyfer,james freedson-jackson,emily althaus,thriller drama,the strange ones
2,simon west,dominic cooper,austin stowell,gemma chan,action thriller,stratton
3,warwick thornton,bryan brown,sam neill,unknown,drama history western,sweet country
4,jaume collet-serra,liam neeson,vera farmiga,patrick wilson,action thriller,the commuter
...,...,...,...,...,...,...
265,etan cohen,will ferrell,john c. reilly,rebecca hall,mystery adventure comedy crime,holmes & watson
266,adam mckay,christian bale,amy adams,steve carell,thriller science fiction action adventure,vice
267,mimi leder,felicity jones,armie hammer,justin theroux,drama history,on the basis of sex
268,karyn kusama,nicole kidman,sebastian stan,toby kebbell,thriller crime drama action,destroyer


In [421]:
final_data['combination'] = final_data['director_name'] + ' ' + final_data['actor_1_name'] + ' ' + final_data['actor_2_name'] + final_data['actor_3_name'] + final_data['genres'] 

In [422]:
final_data

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,combination
0,adam robitel,lin shaye,angus sampson,leigh whannell,horror mystery thriller,insidious: the last key,adam robitel lin shaye angus sampsonleigh whan...
1,lauren wolkstein,alex pettyfer,james freedson-jackson,emily althaus,thriller drama,the strange ones,lauren wolkstein alex pettyfer james freedson-...
2,simon west,dominic cooper,austin stowell,gemma chan,action thriller,stratton,simon west dominic cooper austin stowellgemma ...
3,warwick thornton,bryan brown,sam neill,unknown,drama history western,sweet country,warwick thornton bryan brown sam neillunknownd...
4,jaume collet-serra,liam neeson,vera farmiga,patrick wilson,action thriller,the commuter,jaume collet-serra liam neeson vera farmigapat...
...,...,...,...,...,...,...,...
265,etan cohen,will ferrell,john c. reilly,rebecca hall,mystery adventure comedy crime,holmes & watson,etan cohen will ferrell john c. reillyrebecca ...
266,adam mckay,christian bale,amy adams,steve carell,thriller science fiction action adventure,vice,adam mckay christian bale amy adamssteve carel...
267,mimi leder,felicity jones,armie hammer,justin theroux,drama history,on the basis of sex,mimi leder felicity jones armie hammerjustin t...
268,karyn kusama,nicole kidman,sebastian stan,toby kebbell,thriller crime drama action,destroyer,karyn kusama nicole kidman sebastian stantoby ...


In [423]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2019"
df1 = pd.read_html(link, header=0)[2]
df2 = pd.read_html(link, header=0)[3]
df3 = pd.read_html(link, header=0)[4]
df4 = pd.read_html(link, header=0)[5]

In [424]:
data = df1.append(df2.append(df3.append(df4,ignore_index=True),ignore_index=True),ignore_index=True)

In [425]:
data['genres'] = data['Title'].map(lambda x:get_genre(x))

In [426]:
data['director_name'] = data['Cast and crew'].map(lambda x:get_director(x))
data['actor_1_name'] = data['Cast and crew'].map(lambda x:get_actor1(x))
data['actor_2_name'] = data['Cast and crew'].map(lambda x:get_actor2(x))
data['actor_3_name'] = data['Cast and crew'].map(lambda x:get_actor3(x))

In [427]:
data_2019 = data[['director_name','actor_1_name','actor_2_name','actor_3_name','genres','Title']]

In [428]:
data_2019.rename(columns={'Title':'movie_title'},inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [429]:
data_2019['combination'] = data_2019['actor_1_name'] + ' ' + data_2019['actor_2_name'] + ' '+ data_2019['actor_3_name'] + ' '+ data_2019['director_name'] +' ' + data_2019['genres']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [430]:
data_2019

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,combination
0,Adam Robitel,Taylor Russell,Logan Miller,Deborah Ann Woll,Horror Thriller Mystery,Escape Room,Taylor Russell Logan Miller Deborah Ann Woll A...
1,Jen McGowan,Hermione Corfield,Jay Paulson,Sean O'Bryan,Thriller Drama,Rust Creek,Hermione Corfield Jay Paulson Sean O'Bryan Jen...
2,Wilson Coneybeare,Donald Sutherland,Vincent Kartheiser,Oliver Dennis,Thriller,American Hangman,Donald Sutherland Vincent Kartheiser Oliver De...
3,Charles Martin Smith,Bryce Dallas Howard,Edward James Olmos,Alexandra Shipp,Drama Adventure Family,A Dog's Way Home,Bryce Dallas Howard Edward James Olmos Alexand...
4,Neil Burger,Bryan Cranston,Kevin Hart,Nicole Kidman,Comedy Drama,The Upside,Bryan Cranston Kevin Hart Nicole Kidman Neil B...
...,...,...,...,...,...,...,...
237,"Nick Bruno, Troy Quane",Will Smith,Tom Holland,Rashida Jones,Animation Action Adventure Comedy Family,Spies in Disguise,Will Smith Tom Holland Rashida Jones Nick Brun...
238,Greta Gerwig,Saoirse Ronan,Emma Watson,Florence Pugh,Drama Romance,Little Women,Saoirse Ronan Emma Watson Florence Pugh Greta ...
239,Sam Mendes,George MacKay,Dean-Charles Chapman,Mark Strong,War Drama Action Thriller,1917,George MacKay Dean-Charles Chapman Mark Strong...
240,Destin Daniel Cretton,Michael B. Jordan,Jamie Foxx,Brie Larson,Drama Crime History,Just Mercy,Michael B. Jordan Jamie Foxx Brie Larson Desti...


In [431]:
# for 2020

In [432]:
import bs4 as bs
import urllib.request

In [433]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2020"
source = urllib.request.urlopen(link).read()
soup = bs.BeautifulSoup(source,'lxml')
tables = soup.find_all('table',class_='wikitable sortable')
type(tables[0])
df1 = pd.read_html(str(tables[0]))[0]
df2 = pd.read_html(str(tables[1]))[0]
df3 = pd.read_html(str(tables[2]))[0]
df4 = pd.read_html(str(tables[3]).replace("'1\"\'",'"1"'))[0] # avoided "ValueError: invalid literal for int() with base 10: '1"'


In [434]:
data = df1.append(df2.append(df3.append(df4,ignore_index=True),ignore_index=True),ignore_index=True)

In [435]:
data['genres'] = data['Title'].map(lambda x:get_genre(x))
data['director_name'] = data['Cast and crew'].map(lambda x:get_director(str(x)))
data['actor_1_name'] = data['Cast and crew'].map(lambda x:get_actor1(str(x)))
data['actor_2_name'] = data['Cast and crew'].map(lambda x:get_actor2(str(x)))
data['actor_3_name'] = data['Cast and crew'].map(lambda x:get_actor3(str(x)))

In [436]:
data_2020 = data[['director_name','actor_1_name','actor_2_name','actor_3_name','genres','Title']]

In [437]:
data_2020.rename(columns={'Title':'movie_title'},inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [438]:
data_2020.isna().sum()

director_name    0
actor_1_name     0
actor_2_name     0
actor_3_name     0
genres           1
movie_title      0
dtype: int64

In [439]:
data_2020.dropna(how='any',inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [440]:
data_2020.isna().sum()


director_name    0
actor_1_name     0
actor_2_name     0
actor_3_name     0
genres           0
movie_title      0
dtype: int64

In [441]:
data_2020['combination'] = data_2020['actor_1_name'] + ' ' + data_2020['actor_2_name'] + ' '+ data_2020['actor_3_name'] + ' '+ data_2020['director_name'] +' ' + data_2020['genres']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [442]:
final_data.shape

(270, 7)

In [443]:
data_2019.shape

(242, 7)

In [444]:
data_2020.shape

(272, 7)

In [445]:
data_2020

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,combination
0,Nicolas Pesce,Andrea Riseborough,Demián Bichir,John Cho,Horror Mystery Thriller,The Grudge,Andrea Riseborough Demián Bichir John Cho Nico...
1,William Eubank,Kristen Stewart,Vincent Cassel,Jessica Henwick,Action Horror Science Fiction Thriller,Underwater,Kristen Stewart Vincent Cassel Jessica Henwick...
2,Miguel Arteta,Tiffany Haddish,Rose Byrne,Salma Hayek,Comedy,Like a Boss,Tiffany Haddish Rose Byrne Salma Hayek Miguel ...
3,Jon Avnet,Richard Gere,Peter Dinklage,Walton Goggins,Drama,Three Christs,Richard Gere Peter Dinklage Walton Goggins Jon...
4,Anthony Jerjen,Josh Hartnett,Margarita Levieva,Chandler Riggs,Crime Thriller Drama,Inherit the Viper,Josh Hartnett Margarita Levieva Chandler Riggs...
...,...,...,...,...,...,...,...
268,Robert Rodriguez,Priyanka Chopra Jonas,Pedro Pascal,YaYa Gosselin,Action Fantasy Family Comedy,We Can Be Heroes,Priyanka Chopra Jonas Pedro Pascal YaYa Gossel...
269,Paul Greengrass,Tom Hanks,Helena Zengel,unknown,Adventure Western,News of the World,Tom Hanks Helena Zengel unknown Paul Greengras...
270,Regina King,Kingsley Ben-Adir,Eli Goree,Aldis Hodge,Drama,One Night in Miami...,Kingsley Ben-Adir Eli Goree Aldis Hodge Regina...
271,Emerald Fennell,Carey Mulligan,Bo Burnham,Alison Brie,Thriller Crime Drama,Promising Young Woman,Carey Mulligan Bo Burnham Alison Brie Emerald ...


In [446]:
# data for 2021 movies

In [447]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2021"
source = urllib.request.urlopen(link).read()
soup = bs.BeautifulSoup(source,'lxml')
tables = soup.find_all('table',class_='wikitable sortable')
type(tables[0])
df1 = pd.read_html(str(tables[0]))[0]
df2 = pd.read_html(str(tables[1]))[0]
df3 = pd.read_html(str(tables[2]))[0]
df4 = pd.read_html(str(tables[3]).replace("'1\"\'",'"1"'))[0] # avoided "ValueError: invalid literal for int() with base 10: '1"'

In [448]:
data = df1.append(df2.append(df3.append(df4,ignore_index=True),ignore_index=True),ignore_index=True)
data['genres'] = data['Title'].map(lambda x:get_genre(str(x)))
data['director_name'] = data['Cast and crew'].map(lambda x:get_director(str(x)))
data['actor_1_name'] = data['Cast and crew'].map(lambda x:get_actor1(str(x)))
data['actor_2_name'] = data['Cast and crew'].map(lambda x:get_actor2(str(x)))
data['actor_3_name'] = data['Cast and crew'].map(lambda x:get_actor3(str(x)))

In [449]:
data_2021 = data[['director_name','actor_1_name','actor_2_name','actor_3_name','genres','Title']]
data_2021.rename(columns={'Title':'movie_title'},inplace=True)
data_2021['combination'] = data_2021['actor_1_name'] + ' ' + data_2021['actor_2_name'] + ' '+ data_2021['actor_3_name'] + ' '+ data_2021['director_name'] +' ' + data_2021['genres']

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [450]:
data_2021.isna().sum()

director_name    0
actor_1_name     0
actor_2_name     0
actor_3_name     0
genres           1
movie_title      1
combination      1
dtype: int64

In [451]:
data_2021.dropna(how='any',inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [452]:
data_2021

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,combination
0,Roseanne Liang,Chloë Grace Moretz,Taylor John Smith,Beulah Koale,Horror Action War,Shadow in the Cloud,Chloë Grace Moretz Taylor John Smith Beulah Ko...
1,Ramin Bahrani,Adarsh Gourav,Rajkummar Rao,Priyanka Chopra Jonas,Crime Drama,The White Tiger,Adarsh Gourav Rajkummar Rao Priyanka Chopra Jo...
2,Doug Liman,Anne Hathaway,Chiwetel Ejiofor,Stephen Merchant,Comedy Crime Drama,Locked Down,Anne Hathaway Chiwetel Ejiofor Stephen Merchan...
3,Simon Stone,Carey Mulligan,Ralph Fiennes,Lily James,Drama History,The Dig,Carey Mulligan Ralph Fiennes Lily James Simon ...
4,Mikael Håfström,Anthony Mackie,Damson Idris,Emily Beecham,Thriller Action Science Fiction,Outside the Wire,Anthony Mackie Damson Idris Emily Beecham Mika...
...,...,...,...,...,...,...,...
333,Garth Jennings,Matthew McConaughey,Reese Witherspoon,Scarlett Johansson,Family Comedy Animation Drama Music,Sing 2,Matthew McConaughey Reese Witherspoon Scarlett...
334,Matthew Vaughn,Ralph Fiennes,Gemma Arterton,Rhys Ifans,War Action Thriller Adventure,The King's Man,Ralph Fiennes Gemma Arterton Rhys Ifans Matthe...
335,Joel Coen,Denzel Washington,Frances McDormand,Corey Hawkins,Drama Adventure Thriller War Fantasy,The Tragedy of Macbeth,Denzel Washington Frances McDormand Corey Hawk...
336,Erwin brothers,Zachary Levi,Anna Paquin,Dennis Quaid,Drama,American Underdog,Zachary Levi Anna Paquin Dennis Quaid Erwin br...


In [453]:
web_data = final_data.append(data_2019.append(data_2020.append(data_2021)))

In [454]:
old = pd.read_csv('/content/drive/MyDrive/Movie Recommendation System/Data processing /new_data.csv')

In [455]:
old

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,combination
0,james cameron,cch pounder,joel david moore,wes studi,action adventure fantasy sci-fi,Avatar,james cameron cch pounder joel david moorewes ...
1,gore verbinski,johnny depp,orlando bloom,jack davenport,action adventure fantasy,Pirates of the Caribbean: At World's End,gore verbinski johnny depp orlando bloomjack d...
2,sam mendes,christoph waltz,rory kinnear,stephanie sigman,action adventure thriller,Spectre,sam mendes christoph waltz rory kinnearstephan...
3,christopher nolan,tom hardy,christian bale,joseph gordon-levitt,action thriller,The Dark Knight Rises,christopher nolan tom hardy christian balejose...
4,doug walker,doug walker,rob walker,unknown,documentary,Star Wars: Episode VII - The Force Awakens ...,doug walker doug walker rob walkerunknowndocum...
...,...,...,...,...,...,...,...
5373,jim strouse,jessica williams,chris o'dowd,keith stanfield,romance comedy,the incredible jessica james,jim strouse jessica williams chris o'dowdkeith...
5374,farhad mann,adelaide kane,benjamin hollingsworth,jean louisa kelly,romance,can't buy my love,farhad mann adelaide kane benjamin hollingswor...
5375,hannaleena hauru,inka haapamäki,rosa honkonen,tiitus rantala,romance comedy,thick lashes of lauri mäntyvaara,hannaleena hauru inka haapamäki rosa honkonent...
5376,jonathan a. rosenbaum,lou diamond phillips,wallace shawn,gina holden,crime comedy action family,cop and a half: new recruit,jonathan a. rosenbaum lou diamond phillips wal...


In [456]:
final_df = old.append(web_data)

In [457]:
final_df

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,combination
0,james cameron,cch pounder,joel david moore,wes studi,action adventure fantasy sci-fi,Avatar,james cameron cch pounder joel david moorewes ...
1,gore verbinski,johnny depp,orlando bloom,jack davenport,action adventure fantasy,Pirates of the Caribbean: At World's End,gore verbinski johnny depp orlando bloomjack d...
2,sam mendes,christoph waltz,rory kinnear,stephanie sigman,action adventure thriller,Spectre,sam mendes christoph waltz rory kinnearstephan...
3,christopher nolan,tom hardy,christian bale,joseph gordon-levitt,action thriller,The Dark Knight Rises,christopher nolan tom hardy christian balejose...
4,doug walker,doug walker,rob walker,unknown,documentary,Star Wars: Episode VII - The Force Awakens ...,doug walker doug walker rob walkerunknowndocum...
...,...,...,...,...,...,...,...
333,Garth Jennings,Matthew McConaughey,Reese Witherspoon,Scarlett Johansson,Family Comedy Animation Drama Music,Sing 2,Matthew McConaughey Reese Witherspoon Scarlett...
334,Matthew Vaughn,Ralph Fiennes,Gemma Arterton,Rhys Ifans,War Action Thriller Adventure,The King's Man,Ralph Fiennes Gemma Arterton Rhys Ifans Matthe...
335,Joel Coen,Denzel Washington,Frances McDormand,Corey Hawkins,Drama Adventure Thriller War Fantasy,The Tragedy of Macbeth,Denzel Washington Frances McDormand Corey Hawk...
336,Erwin brothers,Zachary Levi,Anna Paquin,Dennis Quaid,Drama,American Underdog,Zachary Levi Anna Paquin Dennis Quaid Erwin br...


In [458]:
# converting all str to lower
final_df['director_name'] = final_df['director_name'].str.lower()
final_df['actor_1_name'] = final_df['actor_1_name'].str.lower()
final_df['actor_2_name'] = final_df['actor_2_name'].str.lower()
final_df['actor_3_name'] = final_df['actor_3_name'].str.lower()
final_df['genres'] = final_df['genres'].str.lower()
final_df['movie_title'] = final_df['movie_title'].str.lower()
final_df['combination'] = final_df['combination'].str.lower()
final_df

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,combination
0,james cameron,cch pounder,joel david moore,wes studi,action adventure fantasy sci-fi,avatar,james cameron cch pounder joel david moorewes ...
1,gore verbinski,johnny depp,orlando bloom,jack davenport,action adventure fantasy,pirates of the caribbean: at world's end,gore verbinski johnny depp orlando bloomjack d...
2,sam mendes,christoph waltz,rory kinnear,stephanie sigman,action adventure thriller,spectre,sam mendes christoph waltz rory kinnearstephan...
3,christopher nolan,tom hardy,christian bale,joseph gordon-levitt,action thriller,the dark knight rises,christopher nolan tom hardy christian balejose...
4,doug walker,doug walker,rob walker,unknown,documentary,star wars: episode vii - the force awakens ...,doug walker doug walker rob walkerunknowndocum...
...,...,...,...,...,...,...,...
333,garth jennings,matthew mcconaughey,reese witherspoon,scarlett johansson,family comedy animation drama music,sing 2,matthew mcconaughey reese witherspoon scarlett...
334,matthew vaughn,ralph fiennes,gemma arterton,rhys ifans,war action thriller adventure,the king's man,ralph fiennes gemma arterton rhys ifans matthe...
335,joel coen,denzel washington,frances mcdormand,corey hawkins,drama adventure thriller war fantasy,the tragedy of macbeth,denzel washington frances mcdormand corey hawk...
336,erwin brothers,zachary levi,anna paquin,dennis quaid,drama,american underdog,zachary levi anna paquin dennis quaid erwin br...


In [459]:
final_df.isna().sum()

director_name    0
actor_1_name     0
actor_2_name     0
actor_3_name     0
genres           6
movie_title      0
combination      3
dtype: int64

In [460]:
final_df.dropna(how='any',inplace=True)

In [461]:
final_df.isna().sum()

director_name    0
actor_1_name     0
actor_2_name     0
actor_3_name     0
genres           0
movie_title      0
combination      0
dtype: int64

In [462]:
final_df.to_csv('main_data.csv',index=False)